# Init vars and import libs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

In [ ]:
from datetime import date

In [ ]:
import numpy as np
import re
import json

# Preprocessing functions

In [ ]:
def bdate_to_age(bdate):
  #if isinstance(bdate, float):
  #  return None
  try:
    d, m, y = bdate.split('.')
    d, m, y = int(d), int(m), int(y)
    today = date.today()
    age = today.year - y - ((today.month, today.day) < (m, d))
  except:
    age = None
  return age

In [ ]:
def value_to_bool(val):
  if isinstance(val, float):
    return 0 if np.isnan(val) else 1
  if val:
    return 1
  else:
    return 0

In [ ]:
def list_to_bool(val):
  if isinstance(val, float):
    return 0 if np.isnan(val) else 1
  if val.replace('[','').replace(']',''):
    return 1
  else:
    return 0

In [ ]:
def leave_only_letters(sentence):
  if isinstance(sentence, str):
    sentence = re.sub('[^a-zа-яё]', ' ', sentence.lower())
    sentence = re.sub(' +', ' ', sentence)
    if sentence == ' ':
      return None
    return sentence
  return None

In [ ]:
def occupation_to_type(value):
  # type = work, school or university = 0, 1, 2 + 3 (unknown)
  try:
    t = json.loads(value.replace("\'",'"').lower())['type']
    if t == 'work':
      return 0
    return 1 if t == 'school' else 2
  except:
    return 3

In [ ]:
def personal_to_attitudes(value, att):
  try:
    t = json.loads(value.replace("\'",'"').lower())[att]
    return t
  except:
    return None

In [ ]:
def prepare_db(df, add_column):

  #add new columns
  for new_cols in add_column:
    print(new_cols[0], new_cols[1])  
    df[new_cols[0]] = new_cols[1]
  
  #delete columns
  df = df.drop(['first_name', 'id', 'friend_status', 'last_name', 'Unnamed: 0', 'mobile_phone', 'home_phone'], axis=1)

  # bdate to age
  df['age'] = df['bdate'].apply(bdate_to_age) 
  df = df.drop(['bdate'], axis=1)

  # change from 1.0\Nan to 1\0
  df['has_mobile'] = df['has_mobile'].apply(value_to_bool)
  df['can_see_all_posts'] = df['can_see_all_posts'].apply(value_to_bool)
  df['can_see_audio'] = df['can_see_audio'].apply(value_to_bool)
  df['can_send_friend_request'] = df['can_send_friend_request'].apply(value_to_bool)
  df['can_be_invited_group'] = df['can_be_invited_group'].apply(value_to_bool)

  # text to bool
  df['interests'] = df['interests'].apply(value_to_bool)
  df['books'] = df['books'].apply(value_to_bool)
  df['quotes'] = df['quotes'].apply(value_to_bool)
  df['about'] = df['about'].apply(value_to_bool)
  df['music'] = df['music'].apply(value_to_bool)
  df['movies'] = df['movies'].apply(value_to_bool)
  df['interests'] = df['interests'].apply(value_to_bool)

  # status - leave only letters(words)
  df['status'] = df['status'].apply(leave_only_letters)
  # if status exists
  df['is_status'] = df['status'].apply(value_to_bool)

  # age, followers_count - to average
  df['followers_count'] = df['followers_count'].fillna(df['followers_count'].mean())
  df['age'] = df['age'].fillna(df['age'].mean())

  # occupation - 0/1/2/3(unknown) (depends on type)
  df['occupation'] = df['occupation'].apply(occupation_to_type)

  # delete career university	university_name	faculty	faculty_name	graduation,  use only universities
  df = df.drop(['career', 'university', 'university_name', 'faculty', 'graduation', 'faculty_name'], axis=1)

  # relation
  df['relation'] = df['relation'].fillna(0.0)

  # personal - trying to find attitude to alcohol and smoking
  # 0 - unknown
  df['alcohol'] = df['personal'].apply(personal_to_attitudes, args=('alcohol',)).fillna(0.0)
  df['smoking'] = df['personal'].apply(personal_to_attitudes, args=('smoking',)).fillna(0.0)
  df = df.drop(['personal'], axis=1)

  # if exists
  df['universities'] = df['universities'].apply(list_to_bool)
  df['schools'] = df['schools'].apply(list_to_bool)
  df['relatives'] = df['relatives'].apply(list_to_bool)

  # clean data from non-active users
  df = df[df.deactivated != 'banned']
  df = df[df.deactivated != 'deleted']
  df = df.drop(['deactivated'], axis=1)

  df['skype'] = df['skype'].apply(value_to_bool)
  df['relation_partner'] = df['relation_partner'].apply(value_to_bool)
  df['twitter'] = df['twitter'].apply(value_to_bool)
  df['instagram'] = df['instagram'].apply(value_to_bool)
  df['education_form'] = df['education_form'].apply(value_to_bool)
  df['education_status'] = df['education_status'].apply(value_to_bool)
  df['facebook'] = df['facebook'].apply(value_to_bool)
  df['facebook_name'] = df['facebook_name'].apply(value_to_bool)
  df['status_audio'] = df['status_audio'].apply(value_to_bool)
  df['livejournal'] = df['livejournal'].apply(value_to_bool)

  return df

In [ ]:
#df = pd.read_csv('/content/drive/My Drive/Course_Project_BD/rap_data/maxkorzh_data.csv',  engine='python' )

In [ ]:
#df = prepare_db(df, add_column = [('category', 'rock_data')])

# Applying for datasets

In [ ]:
genres_dict = {'electro_data':0, 'pop_data':1, 'rap_data':2, 'rock_data':3}

In [ ]:
shapes = dict()

In [ ]:
!ls '/content/drive/My Drive/Course_Project_BD/'

data_downloading.zip  data_pop.json  data_rock.json  pop_data  rock_data
data_electro.json     data_rap.json  electro_data    rap_data


In [ ]:
path_to_data = '/content/drive/My Drive/Course_Project_BD/'

In [ ]:
%%time
for root, dir, files in os.walk(path_to_data+'/rap_data'):
  if root.endswith(tuple(genres_dict.keys())):
    print(os.path.basename(root))
    for file in files:
      print('\t', file)
      #prepare_db(root + '/' + file, add_column = [('category', os.path.basename(root))])
      pd_data = pd.read_csv(root + '/' + file,  engine='python' )
      pd_data = prepare_db(pd_data, add_column = [('category', genres_dict[os.path.basename(root)])])
      shapes[file] = pd_data.shape
      pd_data.to_csv(root + '/2' + file)